# Work through the reduction of a single dataset

In [7]:
# imports
import os

from cosredux import utils 

## Setup files
    Copy files from Dropbox to local, working folder
    cd 'working_folder'
    # Darks, if needed
    cp -rp ~Dropbox/COS-LRG/darksall .
    # Calibs, as needed
    cp -rp ~Dropbox/COS-LRG/tests/calibs .
    # Subset of the raw and object calibration files
    mkdir LCYA010101
    cd LCYA010101
    cp -rp ~Dropbox/COS-LRG/LCYA01010/*rawtag* .
    cp -rp ~Dropbox/COS-LRG/LCYA01010/lcya01010_asn.fits .
    cp -rp ~Dropbox/COS-LRG/LCYA01010/lcya01010_trl.fits .
    cp -rp ~Dropbox/COS-LRG/LCYA01010/lcya01010_j*.fits .

In [15]:
rdx_path = '/Users/xavier/HST/COS/LRG_Redux/'
science_folder = 'LCYA010101/'
dark_folder = 'darks/'
calib_folder = 'calibs/'

## Customize for calcos
    Science frames
    LP2 calib file

### Science frames

In [16]:
utils.modify_rawtag_for_calcos(rdx_path+science_folder)

Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01fyq_rawtag_a.fits
Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01fyq_rawtag_b.fits
Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01g0q_rawtag_a.fits
Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01g0q_rawtag_b.fits
Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01ggq_rawtag_a.fits
Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01ggq_rawtag_b.fits
Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01gjq_rawtag_a.fits
Modifying header cards for rawtag file: /Users/xavier/HST/COS/LRG_Redux/LCYA010101/lcya01gjq_rawtag_b.fits


### Calibs -- May not need to repeat this (i.e. if done before)

In [18]:
utils.modify_LP2_1dx_calib(rdx_path+calib_folder)

----

## Run calcos
    Launch pyraf